**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


In [2]:
import skvideo
skvideo.setFFmpegPath("C:\\FFmpeg\\bin")

In [3]:
import skvideo.io

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [4]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [5]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

###(you can use Markdown and Latex)
1.1 The function act performed took decsions among 4 alternatives, based on the current state. In order to avoid the overfit of the model, some randomness is introduced. Contrary to other classical ML tasks our model continues to learn even during perfomance. It refers also to the function learnt act which choose the action according to the state. In term of randomness we have random action also in the funtion act, which introduced when a specific conditions are met.
1.2 Such result is achieved by parameter epsilon, which is accounted to the introduction of randomness, if a random number is less than $\epsilon$ then agent take a random action in order to explore enviornment. As the process of recieving the reward might be stochastic, taking the same action might lead to different result, and, which is more important, the result could have a mathematical expectation, but because of low number of observation we can not extract the true value of this expectation (may be, by chance the true reward is 1 of this action at specific state, meanwhile, by chance we recieved only zero in the same situation in the past, so we give up this action and lose much reward). 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [6]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [7]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=11 # set small when debugging
epochs_test=3 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))


__Question 2__ Explain the use of the arrays ```position``` and ```board```.

Position is a matrix where agents cosider its personal position as 1, zeros as our possible moves, and -1 the zones we can not achieve.
The board is the matrix of the same size but it contains the information about reward (bonus/malus) for a certain cell.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [8]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        random_result = np.random.randint(0, self.n_action, size=1)[0]
        return random_result

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [9]:

def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        s = env.reset()
        game_over = False
        win = 0
        lose = 0
        while game_over == False:
            a = agent.act(s)
            p_s = s
            s, r, game_over = env.act(a)
            if r > 0:
                win = win + r
            if r < 0:
                lose = lose -r
            loss = agent.reinforce(p_s, s,  a, r, game_over)
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [10]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 6.5/16.0. Average score (-9.5)
Win/lose count 7.5/12.0. Average score (-7.0)
Win/lose count 11.0/17.0. Average score (-6.666666666666667)
Final score: -6.666666666666667


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




First proof:
We have \begin{align}Q^\pi(s,a)&=E{p^{\pi}}[\sum_{0 \leq t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
                 &=r(s,a) + E_{p^{\pi}}[\sum_{1 \leq t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
                 &=r(s,a) + \sum_{(s', a')}..\sum_{} \big( \sum_{1 \leq t} \gamma^{t} r(s_{t}, a_{t})\big) p(s_{1} = s',a_{1} =a', s_2 =... |s_{0}=s,a_{0}=a)
\end{align} Also, knowing that $\pi$ is a markov decision process, it follows markov properties in terms of formulas:
\begin{equation} p(s_{1} = s', a_{1} = a', s_{2} =... |s_{0}=s,a_{0}=a) = p(s_2 ... |s_{0}=s,a_{0}=a, s_{1}=s',a_{1}=a') p(s_{1}=s',a_{1}=a' | s_{0}=s,a_{0}=a) = p(s_2 ... |s_{1}=s',a_{1}=a') p(s_{1}=s',a_{1}=a' | s_{0}=s,a_{0}=a) \end{equation}
So: \begin{align}Q^\pi(s,a) \\&= r(s,a) + \gamma \sum{(s', a')} p(s_{1}=s',a_{1}=a' | s_{0}=s,a_{0}=a) \sum_{(s_2, a_2)}..\sum{} \sum{1 \leq t} \gamma^{t-1} r(s_{t}, a_{t}) p(s_2 ... |s_{1}=s',a_{1}=a') \\
&= r(s,a) + \gamma \sum{(s', a')} p(s_{1}=s',a_{1}=a' | s_{0}=s,a_{0}=a)E{p^{\pi}}[ \sum{1 \leq t} \gamma^{t-1} r(s{t}, a{t})|s_{1}=s',a_{1}=a')] \\ 
&= r(s,a) + \gamma \sum{(s', a')} p(s_{1}=s',a_{1}=a' | s_{0}=s,a_{0}=a) Q^{\pi}(s', a') \\
&= r(s,a) + \gamma E_{(s',a')\sim p(.|s,a)} [Q^{\pi}(s', a')] \end{align}

We have to show that: \begin{equation} Q^* (s,a) = r(s,a) + \gamma \sum{s'} p(s'|s,a) \max{a'} Q^\pi (s',a') \end{equation} We have: \begin{align} Q^* (s,a) &= \max{\pi^*} [r(s,a) + \gamma \sum{(s', a')} p(s', a' | s, a) Q^*(s', a')] \
   &= r(s,a) + \gamma \max_{\pi*} [\sum_{(s', a')} p(s', a' | s, a) Q^*(s', a')]
\end{align} It can be seen that changing the sum and maximums we have an inequality : \begin{align} \max \limits{\pi} [\sum \limits{(s', a')} p(s', a' | s, a) Q^*(s', a')] \leq \sum \limits{(s', a')} p(s', a' | s, a) \max \limits{\pi^*} Q^*(s', a') \end{align}



Now, denoting  $\hat{\pi}(s',a') = \arg\max_{\pi'} Q^{\pi'}(s',a')$
\begin{align} \sum{(s'} p(s_{t+1}=s'|s_t = s, a_t = a ) \max{\pi'}Q^{\pi'}(s',a')\\
&= \sum{(s')} p(s{t+1}=s' |s_t = s, at = a ) Q^{\hat{\pi}(s',a')}(s',a') \\
&\leq \max{\pi'} \sum{s'} p(s{t+1}=s' |s_t = s, a_t = a )Q^{\pi'}(s',a') \end{align}

That's allows us to transform our equation:
\begin{align} Q^{}(s,a)&= r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max{a'}\max{\pi'}Q^{\pi'}(s',a')\\
&= r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max{a'} Q^{}(s',a') \\
&=E{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max{a'}Q^{*}(s',a')]. \end{align}

Finally, our loss function:
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}
As our policy is close to optimal, this function will converge to zero. The higher this residual the farther our policy from optimality.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [11]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) < self.max_memory:
            self.memory.insert(0, m)
        else:
            del self.memory[-1]
            self.memory.insert(0, m)
    def random_access(self):
         return self.memory[np.random.choice(np.arange(0, len(self.memory)))]

***
The pipeline we will use for training is given below:

In [12]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [13]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return(np.argmax(self.model.predict(s.reshape([1,s.shape[0],
                                                      s.shape[1],
                                                      s.shape[2]]))[0,:]))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            target_q[i] =self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))
            if game_over_:
                ######## FILL IN
                input_states[i] = s_
                target_q[i,a_] = r_
            else:
                ######## FILL IN
                input_states[i] = s_
                target_q[i,a_] = r_  + self.discount*max(self.model.predict(n_s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0])
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1, n_nodes = 25,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        # prepare data for fully connected layers
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(n_nodes,activation ='relu'))
        model.add(Dense(n_nodes,activation ='relu'))
        model.add(Dense(n_nodes,activation ='relu'))
        model.add(Dense(n_nodes,activation ='relu'))
        model.add(Dense(4))
        ####### FILL IN
        
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=0.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, 25, prefix='fc_train') # I have put number of epochs manually
HTML(display_videos('fc_train10.mp4'))

Epoch 000/025 | Loss 0.0027 | Win/lose count 3.0/2.0 (1.0)
Epoch 001/025 | Loss 0.0060 | Win/lose count 3.5/10.0 (-6.5)
Epoch 002/025 | Loss 0.0133 | Win/lose count 3.5/3.0 (0.5)
Epoch 003/025 | Loss 0.0074 | Win/lose count 2.5/5.0 (-2.5)
Epoch 004/025 | Loss 0.0050 | Win/lose count 6.5/4.0 (2.5)
Epoch 005/025 | Loss 0.0095 | Win/lose count 0.5/3.0 (-2.5)
Epoch 006/025 | Loss 0.0067 | Win/lose count 2.0/8.0 (-6.0)
Epoch 007/025 | Loss 0.0274 | Win/lose count 2.0/3.0 (-1.0)
Epoch 008/025 | Loss 0.0054 | Win/lose count 5.0/1.0 (4.0)
Epoch 009/025 | Loss 0.0215 | Win/lose count 3.5/2.0 (1.5)
Epoch 010/025 | Loss 0.0017 | Win/lose count 2.0/2.0 (0.0)
Epoch 011/025 | Loss 0.0025 | Win/lose count 1.0/2.0 (-1.0)
Epoch 012/025 | Loss 0.0101 | Win/lose count 3.0/7.0 (-4.0)
Epoch 013/025 | Loss 0.0027 | Win/lose count 3.5/3.0 (0.5)
Epoch 014/025 | Loss 0.0088 | Win/lose count 5.0/2.0 (3.0)
Epoch 015/025 | Loss 0.0007 | Win/lose count 0.5/4.0 (-3.5)
Epoch 016/025 | Loss 0.0063 | Win/lose count 4.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [15]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(64,kernel_size=2,input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(MaxPooling2D(2))
        model.add(BatchNormalization())
        model.add(Conv2D(64,kernel_size = 2,activation='relu'))
        model.add(BatchNormalization())
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [16]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,25,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/025 | Loss 0.1351 | Win/lose count 5.0/7.0 (-2.0)
Epoch 001/025 | Loss 0.4554 | Win/lose count 4.0/1.0 (3.0)
Epoch 002/025 | Loss 0.2904 | Win/lose count 3.5/5.0 (-1.5)
Epoch 003/025 | Loss 0.0468 | Win/lose count 3.0/4.0 (-1.0)
Epoch 004/025 | Loss 0.0359 | Win/lose count 7.0/12.0 (-5.0)
Epoch 005/025 | Loss 0.0538 | Win/lose count 5.0/4.0 (1.0)
Epoch 006/025 | Loss 0.0310 | Win/lose count 2.0/1.0 (1.0)
Epoch 007/025 | Loss 0.0169 | Win/lose count 3.0/3.0 (0.0)
Epoch 008/025 | Loss 0.0158 | Win/lose count 4.0/8.0 (-4.0)
Epoch 009/025 | Loss 0.0156 | Win/lose count 1.5/1.0 (0.5)
Epoch 010/025 | Loss 0.0099 | Win/lose count 3.0/5.0 (-2.0)
Epoch 011/025 | Loss 0.0090 | Win/lose count 4.5/4.0 (0.5)
Epoch 012/025 | Loss 0.0113 | Win/lose count 3.5/1.0 (2.5)
Epoch 013/025 | Loss 0.0243 | Win/lose count 8.0/2.0 (6.0)
Epoch 014/025 | Loss 0.0156 | Win/lose count 9.0/4.0 (5.0)
Epoch 015/025 | Loss 0.0217 | Win/lose count 7.0/5.0 (2.0)
Epoch 016/025 | Loss 0.0943 | Win/lose count 4.0/

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [17]:
env = Environment(grid_size=size, max_time=T,temperature=0.8)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,11,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,11,prefix='fc_test')

Test of the CNN
Win/lose count 5.5/3.0. Average score (2.5)
Win/lose count 7.0/4.0. Average score (2.75)
Win/lose count 10.0/2.0. Average score (4.5)
Win/lose count 4.5/1.0. Average score (4.25)
Win/lose count 9.5/4.0. Average score (4.5)
Win/lose count 16.0/3.0. Average score (5.916666666666667)
Win/lose count 12.5/4.0. Average score (6.285714285714286)
Win/lose count 11.0/7.0. Average score (6.0)
Win/lose count 5.0/0. Average score (5.888888888888889)
Win/lose count 5.0/4.0. Average score (5.4)
Win/lose count 10.5/2.0. Average score (5.681818181818182)
Final score: 5.681818181818182
Test of the FC
Win/lose count 10.0/0. Average score (10.0)
Win/lose count 6.0/6.0. Average score (5.0)
Win/lose count 9.5/1.0. Average score (6.166666666666667)
Win/lose count 9.0/2.0. Average score (6.375)
Win/lose count 3.5/3.0. Average score (5.2)
Win/lose count 9.0/0. Average score (5.833333333333333)
Win/lose count 12.5/4.0. Average score (6.214285714285714)
Win/lose count 9.0/2.0. Average score (6.3

In [18]:
HTML(display_videos('cnn_test10.mp4'))

In [19]:
HTML(display_videos('fc_test10.mp4'))

While on training the Convolutional network considerably outperform the fully-connected network, it seems that on testing their peroformance is not really diffrent. Win-loose score is around 0, probably too much randomness is introduced. Also the higher temperature is accounted to the higher result.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [20]:
# I used almost the same algorithm which was introduced in the notebook for train and environment
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        # Decrease the exploration rate
        agent.set_epsilon(agent.epsilon * 0.5)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, round(win, 2), round(lose, 2), round(win-lose, 2)))
        agent.save(name_weights=prefix+'model.h5', name_model=prefix+'model.json')

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self, e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self, t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        # Bad position
        if train:
            reward = -self.malus_position[self.x, self.y] # Punish the agent if the state was visited before
        self.malus_position[self.x, self.y] = 0.1 # Mark the current state as visited
        # Normal bonus
        reward = reward + self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        
        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        # The state now includes wether the surrounding states were visited before
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
    
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [58]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.7, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, 25, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/025 | Loss 0.2683 | Win/lose count 7.5/23.1 (-15.6)
Epoch 001/025 | Loss 0.1159 | Win/lose count 5.3/23.7 (-18.4)
Epoch 002/025 | Loss 0.1017 | Win/lose count 7.8/23.7 (-15.9)
Epoch 003/025 | Loss 0.0261 | Win/lose count 6.5/20.9 (-14.4)
Epoch 004/025 | Loss 0.0374 | Win/lose count 6.7/21.9 (-15.2)
Epoch 005/025 | Loss 0.0233 | Win/lose count 8.2/21.5 (-13.3)
Epoch 006/025 | Loss 0.0161 | Win/lose count 8.2/22.0 (-13.8)
Epoch 007/025 | Loss 0.0368 | Win/lose count 3.0/21.4 (-18.4)
Epoch 008/025 | Loss 0.0161 | Win/lose count 7.3/25.1 (-17.8)
Epoch 009/025 | Loss 0.0081 | Win/lose count 3.2/21.3 (-18.1)
Epoch 010/025 | Loss 0.0128 | Win/lose count 6.1/20.4 (-14.3)
Epoch 011/025 | Loss 0.0084 | Win/lose count 9.3/21.7 (-12.4)
Epoch 012/025 | Loss 0.0127 | Win/lose count 6.9/22.3 (-15.4)
Epoch 013/025 | Loss 0.0256 | Win/lose count 3.6/19.2 (-15.6)
Epoch 014/025 | Loss 0.0268 | Win/lose count 5.7/20.7 (-15.0)
Epoch 015/025 | Loss 0.0139 | Win/lose count 10.0/20.6 (-10.6)
Epoch 0

In [59]:
# Evaluation
test(agent,env,11,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 4.5/1.0. Average score (3.5)
Win/lose count 1.5/0. Average score (2.5)
Win/lose count 21.5/2.0. Average score (8.166666666666666)
Win/lose count 7.5/1.0. Average score (7.75)
Win/lose count 14.0/1.0. Average score (8.8)
Win/lose count 10.5/1.0. Average score (8.916666666666666)
Win/lose count 3.0/0. Average score (8.071428571428571)
Win/lose count 5.5/2.0. Average score (7.5)
Win/lose count 1.0/1.0. Average score (6.666666666666667)
Win/lose count 4.5/1.0. Average score (6.35)
Win/lose count 4.0/3.0. Average score (5.863636363636363)
Final score: 5.863636363636363


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***